In [76]:
# bringing in necessary packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.neighbors import *
from scipy.stats import zscore

In [77]:
# configure environment

np.set_printoptions(suppress=True)

%matplotlib qt

seed = 8675309
rng = np.random.default_rng(seed)

In [78]:
# reading in data (doesn't really get used, comment out if not)

data_in = "../data/cmu_data.txt"
df = pd.read_csv(data_in, sep = ",", header = 0)
df

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20395,s057,8,46,0.0884,0.0685,-0.0199,0.1095,0.1290,0.0195,0.0945,...,0.1219,0.1383,0.0164,0.0820,0.1329,0.0509,0.1005,0.2054,0.1049,0.1047
20396,s057,8,47,0.0655,0.0630,-0.0025,0.0910,0.1148,0.0238,0.0916,...,0.1008,0.0512,-0.0496,0.1037,0.0868,-0.0169,0.1445,0.2206,0.0761,0.1198
20397,s057,8,48,0.0939,0.1189,0.0250,0.1008,0.1122,0.0114,0.0721,...,0.0913,0.1169,0.0256,0.0689,0.1311,0.0622,0.1034,0.2017,0.0983,0.0905
20398,s057,8,49,0.0923,0.1294,0.0371,0.0913,0.0990,0.0077,0.0992,...,0.0882,0.0821,-0.0061,0.0576,0.0697,0.0121,0.0979,0.1917,0.0938,0.0931


In [79]:
# synthetic data for testing

dims = 2
lims = (0, 3)
num_train = 4
num_test = 2
synth_train = rng.integers(low = lims[0], high = lims[1], size = (num_train, dims))
synth_test = rng.integers(low = lims[0], high = lims[1], size = (num_test, dims))
print(synth_train)
print(synth_test)

[[2 1]
 [1 2]
 [1 1]
 [1 2]]
[[2 0]
 [1 1]]


In [80]:
def z_score_model(train: np.ndarray, test: np.ndarray) -> np.ndarray:
    
   
    score = zscore(test)

    return score

In [81]:
# testing z_score_model 



In [82]:
# anomaly detection metric

def squared_L2_binary_predictor(train: np.ndarray, test: np.ndarray, threshold: float) -> np.ndarray:
    """
    Calculate arithmetic mean of train along axis 1 (model)
    Then compute pairwise distances between test entries and model
    Distances below threshold are cast to 0's (real user) and above threshold are 1's (imposter)

    Args: 
        train: data to form model from
        test: data to compare to model vector
        threshold: barrier between authentic user and imposter

    Returns: vector of length = len(test)
    """
    model = np.mean(train, axis = 0)
    diffs = model[np.newaxis, :] - test
    squared_diffs = np.square(diffs)
    sum_squared_diffs = np.sum(squared_diffs, axis = 1)
    sum_diffs = np.sqrt(sum_squared_diffs)
    imposter_vector = np.where(sum_diffs > threshold, 0, 1)

    return imposter_vector

In [83]:
# testing squared L2

out = squared_L2_binary_predictor(synth_train, synth_test, threshold = 1.5)
print(out)

[0 1]


In [84]:
# organizing data into a matrix separated by subjectID

subjects = df.subject.unique()

dfs_list_by_subject = [df[df.subject == subject] for subject in subjects]

nested_list_by_subject = [df.values.tolist() for df in dfs_list_by_subject]

matrix_by_subject = np.array(nested_list_by_subject)

data = np.copy(matrix_by_subject)

X = data[:, :, 3:]
y = data[:, :, 0]

# X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
# y = y.reshape(y.shape[0]*y.shape[1])

In [85]:
X
# y.shape

array([[['0.1491', '0.3979', '0.2488', ..., '0.3509', '0.2171',
         '0.0742'],
        ['0.1111', '0.3451', '0.234', ..., '0.2756', '0.1917', '0.0747'],
        ['0.1328', '0.2072', '0.0744', ..., '0.2847', '0.1762',
         '0.0945'],
        ...,
        ['0.1642', '0.175', '0.0108', ..., '0.3048', '0.1997', '0.1259'],
        ['0.1623', '0.2126', '0.0503', ..., '0.314', '0.1601', '0.1154'],
        ['0.1792', '0.1889', '0.0097', ..., '0.5261', '0.3862', '0.108']],

       [['0.1333', '0.2636', '0.1303', ..., '0.2997', '0.1994',
         '0.1035'],
        ['0.1307', '0.385', '0.2543', ..., '0.3201', '0.2063', '0.1035'],
        ['0.1143', '0.2752', '0.1609', ..., '0.2883', '0.1656', '0.08'],
        ...,
        ['0.1526', '0.1932', '0.0406', ..., '0.258', '0.1266', '0.0932'],
        ['0.1592', '0.1713', '0.0121', ..., '0.214', '0.1076', '0.1006'],
        ['0.1589', '0.1604', '0.0015', ..., '0.243', '0.1208', '0.0929']],

       [['0.0937', '0.4301', '0.3364', ..., '0.3657',

In [86]:
# Manual cross-validation because pipeline's aren't suited for the statistical methods
# thresholds = [i for i in range(0, 100, 5)] + [i for i in range(100, 501, 50)]
start = 0.5
stop = 24
step = .5
thresholds = [round(i, 2) for i in np.arange(start, stop, step)]

user_scores = {threshold: [] for threshold in thresholds}

for user_num, user in enumerate(X):
    # if user_num == 0: 
    # create object to scale data
    scaler = StandardScaler()
    scaled_user = scaler.fit_transform(user)

    # select 200 of 400 samples to use as train, other 200 go to testing for that user
    random_user_indices = np.arange(400)
    rng.shuffle(random_user_indices)
    train_indices = random_user_indices[:200]
    test_user_indices = random_user_indices[200:]
    train = scaled_user[train_indices]
    test_user = scaled_user[test_user_indices]

    # print("Train")
    # print(train[:10])

    # select 5 of 400 samples from each other user to use as imposter testing
    other_user_nums = np.concatenate((np.arange(51)[:user_num], np.arange(51)[user_num+1:]))
    imposter_indices = rng.choice(400, 5)
    test_imposters = X[other_user_nums][:, imposter_indices].reshape((250, 31)).astype("float64")
    test_imposters = scaler.transform(test_imposters)

    # compare both test_user and test_imposters against train using L2 metric at various thresholds
    for i, threshold in enumerate(thresholds):
        ### temp
        if not i:
            min = 100
            max = 10
        ### temp

        user_pred_labels = squared_L2_binary_predictor(train, test_user, threshold)
        imposter_pred_labels = squared_L2_binary_predictor(train, test_imposters, threshold)

        user_pred_tpr = (np.sum(user_pred_labels) / user_pred_labels.size)
        imposter_pred_fpr = (np.sum(imposter_pred_labels) / imposter_pred_labels.size)

        # print(user_pred_labels)
        # print(user_pred_tpr)

        # print(imposter_pred_labels)
        # print(imposter_pred_fpr)
        # if i < 10:
        #     print(f"Threshold: {threshold}")
        #     print(f"User Pred TPR: {user_pred_tpr}\n")

        user_scores[threshold].append((user_pred_tpr, imposter_pred_fpr))

In [87]:
print(f"Minimum value: {min}\nMaximum value: {max}")

Minimum value: 100
Maximum value: 10


In [88]:
# Generate ROC Curve

user_scores_arr_dict = {key: np.array(value) for key, value in user_scores.items()}

user_scores_avg_dict = {key: np.mean(value, axis = 0).tolist() for key, value in user_scores_arr_dict.items()}

thresholds = [str(key) for key in user_scores_avg_dict.keys()]
tpr_fpr_scores = [val for val in user_scores_avg_dict.values()]

tpr_scores = [item[0] for item in tpr_fpr_scores]
fpr_scores = [item[1] for item in tpr_fpr_scores]

### log tpr and fpr scores for reading
# for t, f in zip(tpr_scores, fpr_scores):
#     print(f"FPR, TPR: {(round(f, 3), round(t, 3))}")

### plotting ROC Curve
fig, ax = plt.subplots()

sns.scatterplot(x = fpr_scores, y = tpr_scores, ax = ax)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")

v, h = .1, .1
ax.set_xlim(0-h, 1+h)
ax.set_ylim(0-v, 1+v)

# Loop through the data points 
for i, threshold in enumerate (thresholds):
    plt.text(fpr_scores[i], tpr_scores[i], threshold)

vals = [i for i in np.arange(0, 1, 0.01)]
xp = [i for i in np.arange(1, 0, -0.01)]

sns.scatterplot(x = vals, y = vals, ax = ax)
sns.scatterplot(x = vals, y = xp, ax = ax)

fig.suptitle("ROC Curve for L2 Anomaly Detector")

plt.show()
###

In [89]:
# Calculate AUC
auc = np.trapz(y = tpr_scores, x = fpr_scores)

print(f"AUC = {auc}")

AUC = 0.8246780622837372
